In [ ]:

from flask import Flask, request, jsonify
from sklearn.preprocessing import LabelEncoder
import requests
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from flask_cors import CORS

import nest_asyncio

nest_asyncio.apply()

app = Flask(__name__)
CORS(app) 

def fetch_companies():
    base_url = 'http://localhost:8023/company/getAll'
    api_url = f'{base_url}'
    response = requests.get(api_url)
    if response.status_code == 200:
        # Extract JSON data from the response
        companiesData = response.json()
        # Print the structure of the data
        return companiesData
    else:
        print(f"Failed request {response.status_code}")
        return None
        
def make_predictions(companyData,best_xgb_model):
    
    if companyData.empty:
        print("Erreur: Empty data.")
        return None
    
    # Supprimer les valeurs manquantes
    companyData.fillna(0, inplace=True)
    
    # Supprimer les colonnes inutiles
    companyData.drop(columns=['activity', 'rib', 'accountType', 'id', 'description'], inplace=True)
    
    # Instancier LabelEncoder
    label_encoder = LabelEncoder()

    # Encoder la colonne 'companyName'
    companyData['companyName'] = label_encoder.fit_transform(companyData['companyName'])

    # Encoder la colonne 'reportedCurrency'
    companyData['reportedCurrency'] = label_encoder.fit_transform(companyData['reportedCurrency'])
    
    # Utiliser le modèle XGBoost pour faire des prédictions
    predictions = best_xgb_model.predict(companyData)

    return predictions   

def preprocess_data_and_split(file_path):
    # Charger les données financières depuis le fichier CSV
    Financial_data = pd.read_csv(file_path)
    
    # Supprimer les lignes contenant des valeurs manquantes
    Financial_data.dropna(inplace=True)
    
    # Supprimer les colonnes inutiles
    Financial_data.drop(columns=['Unnamed: 0', 'risk', 'changeInCashAndCashEquivalents'], inplace=True) 
    
    # Instancier LabelEncoder
    label_encoder = LabelEncoder()
    
    # Encoder la colonne 'companyName'
    Financial_data['companyName'] = label_encoder.fit_transform(Financial_data['companyName'])
    
    # Encoder la colonne 'reportedCurrency'
    Financial_data['reportedCurrency'] = label_encoder.fit_transform(Financial_data['reportedCurrency'])  
    
    Financial_data 
    
    # Séparer les données en X (features) et Y (target)
    X = Financial_data.drop(columns=["profit"])
    Y = Financial_data["profit"]
    
    # Diviser les données en ensembles d'entraînement, de test et de validation
    X_train, X_test_val, Y_train, Y_test_val = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    
    return X_train, X_test_val, Y_train, Y_test_val


def get_trained_xgboost_model(X_train, X_val, Y_train, Y_val):
    params = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [11, 10, 9],
        'min_child_weight': [6, 7, 8],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }

    # Initialize the XGBoost model with default hyperparameters
    xgb_model = xgb.XGBRegressor()

    # Grid search to find the best hyperparameters
    cv_results = GridSearchCV(estimator=xgb_model, param_grid=params, cv=3, scoring='neg_mean_squared_error')
    cv_results.fit(X_train, Y_train)
    best_params = cv_results.best_params_

    # Use the best hyperparameters to train the model
    best_xgb_model = xgb.XGBRegressor(**best_params)
    best_xgb_model.fit(X_train, Y_train)

    # Predict on the validation set and calculate the mean squared error
    Y_pred_xgb = best_xgb_model.predict(X_val)
    mse_xgb = mean_squared_error(Y_val, Y_pred_xgb)
    print("Mean Squared Error on the validation set with XGBoost:", mse_xgb)

    return best_xgb_model


@app.route('/recommend_companies', methods=['POST'])
def recommend_companies():
    
    # Récupérer l'investisseur ID depuis la requête
    investor_id = request.json['investor_id']
    
    
    # Fetch companies data
    companiesData = fetch_companies()
    companiesData
    if companiesData is None:
        return jsonify({'error': 'Failed to fetch companies data.'}), 500
    
    
    # Pretraitement de Financial Data et generation du model XGBoost
    file_path = 'C:\\Users\\Ranim\\Downloads\\financiallll2.csv'
    X_train, X_test_val, Y_train, Y_test_val = preprocess_data_and_split(file_path)
    best_xgb_model = get_trained_xgboost_model(X_train, X_test_val, Y_train, Y_test_val)
 
    if best_xgb_model is None:
        return jsonify({'error': 'Failed to load XGBoost model.'}), 500
    

    # Récupérer la prédiction de tolérance au risque de l'investisseur
    base_url="http://localhost:8023/profileData/findProfile/"
    
    resp=requests.post('http://localhost:5004/predict_tolerance_risk', json={'base_url': base_url, 'investor_id': investor_id}).json()
    tolerance_risk_prediction = resp['predicted_tolerance_risk'][0]
   
    # Générer les recommandations d'entreprises
    recommended_companies_json = []
    max_recommendations = 12
    
    for company in companiesData:
        if len(recommended_companies_json) >= max_recommendations:
            break

        if not company:
            print("Comapnies List is empty.")
            continue

        company_data_df = pd.DataFrame([company])
        activity= company_data_df['activity'].values[0] if not company_data_df.empty else None
        description= company_data_df['description'].values[0] if not company_data_df.empty else None
        print("activity",activity)
        # Utiliser la fonction make_predictions avec les données de cette entreprise
        predicted_profit = make_predictions(company_data_df, best_xgb_model)
        print('predicted_profit',predicted_profit)

        # Vérifier si des prédictions ont été générées
        if predicted_profit :
            
            predicted_profit_float = float(predicted_profit[0])  # Convertir en float
            if predicted_profit_float > 100  :
                predicted_profit_float = 100

            # Filtrer les entreprises en fonction de la tolérance au risque prédite de l'investisseur
            if  0 < tolerance_risk_prediction and tolerance_risk_prediction <= 20:
                # Recommander toutes les entreprises avec un profit de 20% ou plus
                if  80 < predicted_profit_float and predicted_profit_float <= 100  :
                    recommended_companies_json.append({'companyName': company['companyName'], 'profit': predicted_profit_float, 'activity': activity ,'description':description})
                 
            elif 20 < tolerance_risk_prediction and tolerance_risk_prediction <= 40:
                # Recommander les entreprises avec un profit entre 30% et 100%
                if 60 < predicted_profit_float and predicted_profit_float <= 100  :
                    recommended_companies_json.append({'companyName': company['companyName'], 'profit': predicted_profit_float, 'activity': activity, 'description':description})
                  
            elif 40 < tolerance_risk_prediction and tolerance_risk_prediction <= 60  :
                # Recommander les entreprises avec un profit entre 50% et 100%
                if 40 < predicted_profit_float and predicted_profit_float <= 100:
                    recommended_companies_json.append({'companyName': company['companyName'], 'profit': predicted_profit_float , 'activity': activity ,'description':description})
                   
            elif 60 < tolerance_risk_prediction and tolerance_risk_prediction <= 80:
                # Recommander les entreprises avec un profit entre 70% et 100%
                if 20 < predicted_profit_float and predicted_profit_float <= 100  :
                    recommended_companies_json.append({'companyName': company['companyName'], 'profit': predicted_profit_float ,'activity': activity ,'description':description})
                   
            elif 80 < tolerance_risk_prediction and tolerance_risk_prediction <= 100:
                # Recommander les entreprises avec un profit entre 90% et 100%
                if 0 < predicted_profit_float and predicted_profit_float <= 100  :
                    recommended_companies_json.append({'companyName': company['companyName'], 'profit': predicted_profit_float , 'activity': activity , 'description':description})
                    
        else:
            print("pas de recommandation générée pour cette entreprise.")
            
    preffered_activity = requests.post('http://localhost:5004/predict_tolerance_risk', json={'base_url': base_url, 'investor_id': investor_id}).json().get('activity')
    print(preffered_activity)
    filtered_recommended_companies = [company for company in recommended_companies_json if company.get('activity') == preffered_activity]

    return jsonify({'recommended_companies': filtered_recommended_companies}), 200

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/May/2024 22:26:10] "OPTIONS /recommend_companies HTTP/1.1" 200 -


Mean Squared Error on the validation set with XGBoost: 101.54817039817044
activity Finance
predicted_profit [83.37804]
activity Technology
predicted_profit [98.83949]
activity Technology
predicted_profit [97.33631]
activity Technology
predicted_profit [97.33631]
activity Finance
predicted_profit [83.18572]
activity Technology
predicted_profit [85.31423]
activity Technology
predicted_profit [56.94652]
activity Technology
predicted_profit [100.391136]
activity Technology
predicted_profit [98.83949]


127.0.0.1 - - [08/May/2024 22:26:46] "POST /recommend_companies HTTP/1.1" 200 -


Technology


...

Technology


127.0.0.1 - - [09/May/2024 04:20:13] "OPTIONS /recommend_companies HTTP/1.1" 200 -


Mean Squared Error on the validation set with XGBoost: 101.54817039817044
activity Finance
predicted_profit [83.37804]
activity Technology
predicted_profit [98.83949]
activity Technology
predicted_profit [97.33631]
activity Technology
predicted_profit [97.33631]
activity Finance
predicted_profit [83.18572]
activity Technology
predicted_profit [85.31423]
activity Technology
predicted_profit [56.94652]
activity Technology
predicted_profit [100.391136]
activity Technology
predicted_profit [98.83949]


127.0.0.1 - - [09/May/2024 04:20:59] "POST /recommend_companies HTTP/1.1" 200 -


Technology
